In [15]:
import pandas as pd
import numpy as np
import os
import sys
import re
import matplotlib.pyplot as plt
import seaborn as sns
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
from Bio.Alphabet import IUPAC
from matplotlib_venn import venn2
from matplotlib_venn import venn3

In [16]:
def import_VCF4_to_pandas(vcf_file, sep='\t'):
    header_lines = 0
    with open(vcf_file) as f:
        first_line = f.readline().strip()
        next_line = f.readline().strip()
        while next_line.startswith("##"):
            header_lines = header_lines + 1
            #print(next_line)
            next_line = f.readline()
    
    if first_line.endswith('VCFv4.2'):
        
        #Use first line as header
        dataframe = pd.read_csv(vcf_file, sep=sep, skiprows=[header_lines], header=header_lines)
        sample = dataframe.columns[-1]
        dataframe.rename(columns={sample:'sample'}, inplace=True)
        
        for index in dataframe.index:
            info_fields = re.findall(r';*([a-zA-Z]{1,20})=', dataframe.loc[index,'INFO'])
            info_values = re.findall(r'-?\d+\.?\d*e?[+-]?\d{0,2}', dataframe.loc[index,'INFO'])
            
            format_fields = dataframe.loc[index,'FORMAT'].split(":")
            format_values = dataframe.loc[index,'sample'].split(":")
                                    
            for ifield, ivalue in zip(info_fields,info_values):
                dataframe.loc[index,ifield] = ivalue
                
            for ffield, fvalue in zip(format_fields,format_values):
                dataframe.loc[index,ffield] = fvalue
            #if len(format_values[1].split(",")) != 2:
            #    print(format_values[1].split(","), index)
            #    print(dataframe.iloc[index])
        dataframe.rename(columns={'AF':'af'}, inplace=True)
        dataframe['REF_AD'] = dataframe['AD'].str.split(",").str[-2:].str[0]
        dataframe['ALT_AD'] = dataframe['AD'].str.split(",").str[-2:].str[1]
        
        to_float = ['QUAL', 'AC', 'af', 'AN', 'BaseQRankSum', 'DP', 'ExcessHet', 'FS',
       'MLEAC', 'MLEAF', 'MQ', 'MQRankSum', 'QD', 'ReadPosRankSum', 'SOR','GQ','ALT_AD', 'REF_AD']
        
        to_int = ['POS']
        
        for column in dataframe.columns:
            if column in to_float:
                dataframe[column] = dataframe[column].astype(float)
                
        for column in dataframe.columns:
            if column in to_int:
                dataframe[column] = dataframe[column].astype(int)
        
        dataframe['aF'] = dataframe['REF_AD']/dataframe['DP']
        dataframe['AF'] = dataframe['ALT_AD']/dataframe['DP']

                
    else:
        print("This vcf file is not v4.2")
        sys.exit(1)
           
    return dataframe
    

In [17]:
vcf_1 = 'test/1mixed.vcf'
vcf_2 = 'test/2mixed.vcf'
vcf_3 = 'test/3mixed.vcf'
vcf_4 = 'test/4mixed.vcf'
vcf_5 = 'test/5mixed.vcf'

vcf_files = [vcf_1, vcf_2, vcf_3, vcf_4, vcf_5]

#reference="data/MTB_ancestorII_reference.fas"

#fasta_to_replace = SeqIO.read(reference,"fasta")

sample_list = ["1", "2", "3", "4", "5"]
sample_list_df_name = [sample + "_df" for sample in sample_list]

dict_df = {}
for vcf, df_name in zip(vcf_files, sample_list_df_name):
    dict_df[df_name] = import_VCF4_to_pandas(vcf)

FileNotFoundError: [Errno 2] No such file or directory: 'test/1mixed.vcf'

In [ ]:
df1['AD'].str.split(",").str[-2:].str[1]

In [ ]:
df = import_VCF4_to_pandas(vcf)

In [ ]:
vcf = "test/3mixed.snp.hf.vcf"
vcf2 = "test/2mixed.snp.hf.vcf"

In [ ]:
df1.head()

In [ ]:
df1.dtypes

In [ ]:
print(df.columns)